# Advanced Algorithmic Trading Strategy Development in Jupyter Notebook
This Jupyter Notebook outlines a more complex approach to algorithmic trading strategy development. We’ll create an interactive notebook where users can select the stock ticker, start and end dates for data collection, and then develop and test trading strategies. The notebook will use free data sources such as Yahoo Finance and Alpha Vantage.

## Objective
Develop and test sophisticated algorithmic trading strategies using historical market data with interactive user inputs.

## Components
Strategy Design: Implement various trading algorithms such as momentum, mean reversion, and pairs trading.
Backtesting: Evaluate strategy performance with historical data.
Execution: Integrate with trading APIs for live testing or paper trading.
Performance Evaluation: Analyze metrics like Sharpe ratio, alpha, beta, and drawdowns.
Data Sources
Yahoo Finance: Historical price data.
Alpha Vantage: Real-time and historical data.

## Import Libraries and Collect Data

We'll start by importing necessary libraries and collecting historical data using yfinance and Alpha Vantage.

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
from ipywidgets import interact, widgets


In [10]:
# Collect data from Yahoo Finance
def get_data_yahoo(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Collect data from Alpha Vantage
def get_data_alpha_vantage(ticker, api_key):
    ts = TimeSeries(key=api_key, output_format='pandas')
    data, _ = ts.get_daily(symbol=ticker, outputsize='full')
    return data


## Interactive Widgets

Create widgets for user input to select stock ticker and dates.

In [14]:
def interactive_strategy(ticker, start_date, end_date):
    # Get data
    data_yahoo = get_data_yahoo(ticker, start_date, end_date)
    data = data_yahoo[['Close']]
    data['Date'] = data.index
    data.reset_index(drop=True, inplace=True)

    # Calculate Moving Averages
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()

    # Generate signals
    data['Signal'] = 0
    data['Signal'][50:] = np.where(data['SMA_50'][50:] > data['SMA_200'][50:], 1, 0)
    data['Position'] = data['Signal'].diff()

    # Backtesting
    initial_balance = 10000  # Starting balance
    balance = initial_balance
    position = 0
    buy_price = 0

    for i in range(len(data)):
        if data['Position'][i] == 1:  # Buy signal
            position = balance / data['Close'][i]
            buy_price = data['Close'][i]
            balance = 0
        elif data['Position'][i] == -1:  # Sell signal
            balance = position * data['Close'][i]
            position = 0

    # Final balance
    if position > 0:
        balance = position * data['Close'].iloc[-1]

    print(f'Final Balance: ${balance:.2f}')

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(data['Date'], data['Close'], label='Close Price', alpha=0.5)
    plt.plot(data['Date'], data['SMA_50'], label='50-Day Moving Average')
    plt.plot(data['Date'], data['SMA_200'], label='200-Day Moving Average')

    # Plot buy signals
    plt.plot(data[data['Position'] == 1]['Date'], data['SMA_50'][data['Position'] == 1], '^', markersize=10, color='g', label='Buy Signal')

    # Plot sell signals
    plt.plot(data[data['Position'] == -1]['Date'], data['SMA_50'][data['Position'] == -1], 'v', markersize=10, color='r', label='Sell Signal')

    plt.title(f'{ticker} Moving Average Crossover Strategy')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()
    evaluate_performance(data)
# Interactive widgets
interact(interactive_strategy,
         ticker=widgets.Text(value='AAPL', description='Ticker:'),
         start_date=widgets.DatePicker(value=pd.to_datetime('2020-01-01'), description='Start Date:'),
         end_date=widgets.DatePicker(value=pd.to_datetime('2023-01-01'), description='End Date:'))



interactive(children=(Text(value='AAPL', description='Ticker:'), DatePicker(value=Timestamp('2020-01-01 00:00:…

<function __main__.interactive_strategy(ticker, start_date, end_date)>

## Implement Trading Strategy

We’ll use a Moving Average Crossover strategy, which involves two moving averages: a short-term (e.g., 50 days) and a long-term (e.g., 200 days). Buy signals occur when the short-term moving average crosses above the long-term moving average, and sell signals occur when it crosses below.

## Strategy Design and Evaluation

Expand this notebook by adding more strategies and evaluation metrics:

Momentum Strategy: Buy stocks that are trending up and sell those trending down.
Mean Reversion Strategy: Trade based on the assumption that prices will revert to the mean.
Pairs Trading: Identify pairs of stocks with correlated price movements and trade based on divergence.
Add performance metrics for evaluation:

In [12]:
def evaluate_performance(data):
    returns = data['Close'].pct_change()
    strategy_returns = data['Signal'].shift(1) * returns
    cumulative_returns = (1 + strategy_returns).cumprod() - 1
    sharpe_ratio = np.sqrt(252) * strategy_returns.mean() / strategy_returns.std()
    
    print(f'Final Balance: ${balance:.2f}')
    print(f'Sharpe Ratio: {sharpe_ratio:.2f}')
    
    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_returns.index, cumulative_returns, label='Strategy Returns')
    plt.title('Cumulative Returns')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.grid(True)
    plt.show()

# Call this function in the interactive_strategy function for performance evaluation
